# Team Quantimize
## Overview of this notebook:
### Setup:
- Project structure
- Installation
### Usage:
- Data preparation
- Visualization
- Classic solutions
- Quantum solutions
- Air security
- Benchmarks
- Results and discussion

This notebook shall run without any errors or problems and should demonstrate the result of our work, as well as a comparision and a possible quantum advantage.

## Setup
### Project structure
The project is structured the following way:
Quantimize.ipynb is the main entry point. Every file has a brief explanation of what it contains and what it does.
- quantimize/:
    - classic/:
        - classic_solution.py: for the genetic algorithm (GA)
        - toolbox.py:for the straight line solution and various needed methods for the GA
    - data/:
        - aCCF_0623_p_spec.nc: the original atmospheric data file
        - atmo.json: the new created atmospheric data file
        - bada_data.csv: file containing information about the flight level/TAS/ROC/ROD and fuel consumption
        - bada_data.json: file containing the same information but in a different format
        - flights.csv: file containing information about all the flights (start, stop, FL)
        - flights.json: file containing the same information but in a different format
    - quantum/:
        - QGA.py: as the quantum genetic algorithm
        - quantum_solution.py: containing the QAOA algorithm
        - quantum_neural_network.py: containing the quantum neural network
        - toolbox.py: containing additional functions
    - air_security.py: for air security related checks
    - classic_summary.py: for calls to classic functions
    - converter.py: for conversion of units (kts->km/h, minutes->seconds)
    - create_json_data.py: to explain how we generated our datafiles
    - data_access.py: gives access to all the converted data and returns them
    - quantum_summary.py: for calls to quantum functions
    - visualisation.py: for all visualisation related functions
### Installation
The package can be cloned from github using 
```commandline
git clone https://
cd quantimize
```
After that, one needs to install all required packages using
```commandline
pip install requirements.txt
```
Now, everything should be ready and set up to further run this notebook.

## Usage
This section is dedicated to show how the notebook can be used and to show our work and results.
### Data preparation
The data preparation is all related to the file ```create_json_data.py```. It can be used as a standalone application to convert all three given files (atmospheric data, flight data, flight list) into the corresponding .json files. This was done to get a better handle over the files and to adapt the access to our purposes. For example, some flight levels were converted from hPa to FL for simpler calculations and visualisations on. Requirement for the file to run is, that there has to be a folder data (like in this project) in the current working directory. Also the files need to have the specific names for successful conversion. Further explanaition can be found in the file itself. We now want to concentrate more on the work done.
### Notebook preparation
Important imports to make this notebook work as expected

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

### Visualisation
Visualisation is done for a better understanding of the data and of course for nice pictures to look at. To use the visualisation, we first have to import the visualisation part of the package. Also the datetime package, because we need to work with time objects.

In [ ]:
from quantimize import visualisation as vs
from datetime import time as t

We can now access all functions in it. As an example, we want to plot the atmospheric data of a certain flightlevel and time and see how it looks like. We choose FL = 295 and a time of 8 o'clock in the morning.

In [ ]:
vs.make_atmo_map(295, t(8,0,0))

You will get the atmospheric data plotted in different colors, corresponding to the value and normalized to the min and max value available. In the title, you can also see, that there were some assumptions:
1. We have no atmospheric data for FL 295, so we choose the closest, which is FL 300
2. We also do not have atmospheric data for 8 o'clock in the morning. Therefore, we refer to 6 o'clock in the morning.

Regarding the time, we split the time in intervals from 6 to 9 corresponding to the data of 6 o'clock, 9 to 15 corresponding to the data of 12 o'clock and 15 to 21 corresponding to the data of 18 o'clock.

We also can animate the atmospheric data either for a fixed time variing in the FL or vice verca with a fixed FL variing in time. These function do not work in this notebook, because they are repetitive. To see the results, you need to import the package and run the code in a commandline. For completion it is shown here:

In [ ]:
vs.make_animated_atmo_FL_map(t(12))

In [ ]:
vs.make_animated_atmo_day_map(300)

Further visualisation will be presented, when showing flight paths.
### Classic solutions
In our work, we provide two different classic solutions. One being the simplest, the straight line solution, and the other being the genetic algorithm (GA).
#### Straight line solution
The straight line solution, as the name suggests, just computes a straight line from start to end, without any variation. The function takes the flight number (4) and a timestep dt, here 300 seconds, as input.

In [3]:
flight_nr = 17
dt = 300

In [4]:
import quantimize.classic_summary as classic_solution
trajectory_sls = classic_solution.straight_line_solution(flight_nr, dt)

The trajectory can now be plotted as a 2D graph for example. First we create a map and then plot the flight path.

In [ ]:
%matplotlib widget
map = vs.make_map()
vs.scatter_flight_path_on_map(map, trajectory_sls)

In the graph, you can see a dotted line showing the flight path. We can also now calculate the cost.

In [ ]:
cost = classic_solution.compute_cost(trajectory_sls)
print(cost)

The cost has the dimenstion 10e-12 K.

#### Genetic algorithm solution
In the genetic algorithm solution (GA) we use the genetic algorithm to find boundary points and then calculate from these points the optimal flight trajectory.

In [ ]:
report, solution, trajectory_ga = classic_solution.genetic_algorith_solution(flight_nr, dt)

The objective function also contains already the cost of this flight. We can now also plot this again, this time in 3D to see the flight level changes and the atmospheric data. The flight trajectory is also mapped to a uniform time distance for later calculations.

In [ ]:
ax, map = vs.make_3d_map()
vs.plot_flight_path_on_map_3d_with_atmo_as_slices(ax, map, trajectory_ga)

The colors represent the value of the atmospheric data in squares around the flight path point. On the ground you can see the projection of the flight path. It clearly shows, that with variation in the flight height, one can obtain already a much better result than just staying on the same flight level. We can also check the cost, but these should be similar to the objective function.

In [ ]:
cost = classic_solution.compute_cost(trajectory_ga)
print(cost)

The small deviation comes from a different spacing of the points as mentioned above, but is in the same order as the original result.

### Quantum solution
Our quantum solutions consist of three different. The first is the quantum equivalent to the genetic algorithm. The second one ist a quantum neural network and the third one is a mapping of atmospheric data to a qubit grid.
#### Quantum genetic algortihm


In [ ]:
import quantimize.quantum_summary as quantum_solution
trajectory_qga = quantum_solution.quantum_genetic_algorith_solution(flight_nr, dt)

We can again plot this trajectory and compute its cost

In [ ]:
ax, map = vs.make_3d_map()
vs.plot_flight_path_on_map_3d_with_atmo_as_slices(ax, map, trajectory_qga)
cost = classic_solution.compute_cost(trajectory_qga)
print(cost)

As one can see, the algorithm gives an output, which is not as smooth as the classic solution. The climate efficiency is slightly better than for the classic GA.


#### Quantum neural network
The quantum neural network takes already precalculated boundary points and tries to optimize those further. We use the boundary points from the previous classic GA algorithm.

In [ ]:
ctrl_pts = solution['variable']
optimized_ctrl_pts = quantum_solution.quantum_neural_network(flight_nr, 6, ctrl_pts)

In [ ]:
print(optimized_ctrl_pts)

#### Qubit grid

In [ ]:
gc = quantum_solution.sample_grid()
a,b,c = quantum_solution.run_QAOA(gc)

In [ ]:
print(c)

### Air security
Air security is essential for all planes in the air to avoid crashes and to save lives. After a flight trajectory is calculated, one has to check that the route has no steeper turns than 25°, that the plane is at least 10 FL above and below of other planes or has a distance of 5 nm (9,26 km) to the next plane. This can be checked with our air safety functions.

Since our code relys on timestamps, we first want do do an estimation of how close we have to choose our timesteps:
$\\s = 9.26km\\$
$v_{max} = 459kts = 850 km/h\\$
$t = \frac{s}{v_{max}} = 39,2 s\\$

To further enhance this, we reduce the time to 15 s. This fits good in a general scale of time(minutes) and provides us with enough points to detect possible collisions. Therefore we first adapt our trajectories:

In [5]:
import quantimize.air_security as air_safety

trajectory_sls = classic_solution.straight_line_solution(2, 15)
report, solution, trajectory_ga = classic_solution.genetic_algorith_solution(21, 15)
cost_sls = classic_solution.compute_cost(trajectory_sls)
cost_ga = classic_solution.compute_cost(trajectory_ga)
print(cost_sls, cost_ga)


 The best solution found:                                                                           
 [ -8.62114869  -8.14944913   9.78790485  45.49057421  59.80533896
  59.62998859 329.37483245 377.46169909 310.682234   337.03593454
 300.28309488]

 Objective function:
 -19167.69388516968
1335.4189450460253 1079.4960077301284


In [6]:
air_safety.radius_control(trajectory_ga)
air_safety.check_safety([trajectory_sls, trajectory_ga], 15)

2


c:\Users\Jakob\Documents\TUM\Master\4.Semester-QST\Quantum-Challenge\quantimize\air_security.py:216: RuntimeWarning: invalid value encountered in arccos
  theta = np.arccos(np.dot(v1,v2)/(np.linalg.norm(v1)*np.linalg.norm(v2)))*180/np.pi


KeyError: datetime.time(13, 31, 27)

### Benchmarks
#### Flight time and fuel consumption
#### Computation efficiency
#### Climate impact reduction
### Results and discussion